# Module 1 - Programming Assignment

## General Directions

1. You must follow the Programming Requirements outlined on Canvas.
2. The Notebook should be cleanly and fully executed before submission.
3. You should change the name of this file to be your JHED id. For example, `jsmith299.ipynb` although Canvas will change it to something else... :/

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        You should always read the entire assignment before beginning your work, so that you know in advance what the requested output will be and can plan your implementation accordingly.
    </p>
</div>

# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
🌾       plains     1
🌲       forest     3
🪨       hills      5
🐊       swamp      7
🗻       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
🌾🌾🌾🌾🌲🌾🌾
🌾🌾🌾🌲🌲🌲🌾
🌾🗻🗻🗻🌾🌾🌾
🌾🌾🗻🗻🌾🌾🌾
🌾🌾🗻🌾🌾🌲🌲
🌾🌾🌾🌾🌲🌲🌲
🌾🌾🌾🌾🌾🌾🌾
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [369]:
full_world = [
['🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🗻', '🗻', '🗻', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🪨', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨'],
['🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🐊', '🐊', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🌾'],
['🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🌲', '🌲', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🪨', '🌾'],
['🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🪨', '🌾', '🌾'],
['🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🌾', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🐊', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🪨', '🪨', '🗻', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🌾', '🌾', '🪨', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🪨', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🌲', '🌲', '🪨', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🪨', '🪨', '🪨', '🪨', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🪨', '🪨', '🌾', '🐊', '🌾', '🪨', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🌾', '🌾', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🌲', '🌲', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🪨', '🪨', '🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🪨', '🗻', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🪨', '🪨', '🗻', '🗻', '🌾', '🗻', '🗻', '🪨', '🪨', '🐊', '🐊', '🐊', '🐊'],
['🪨', '🗻', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🗻', '🗻', '🪨', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🐊'],
['🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾']
]

## Warning

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7). Yes, there are many ways to do this. I picked this representation because when you look at it, it *looks* like a regular x, y cartesian grid and it's easy to print out.

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics:

In [370]:
test_world = [
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']
]

**what do you expect the policy would be?** Think about it for a bit. This will help you with your programming and debugging.

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [371]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [372]:
COSTS = { '🌾': 1, '🌲': 3, '🪨': 5, '🐊': 7, '🗻': float('inf')}

## Specification

You will implement a function called `a_star_search` that takes the parameters and returns the value as specified below. The return value is going to look like this:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

You should also implement a function called `pretty_print_path`. 
The `pretty_print_path` function prints an ASCII representation of the path generated by the `a_star_search` on top of the terrain map. 
For example, for the test world, it would print this:

```
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
```

using ⏩,⏪,⏫ ⏬ to represent actions and `🎁` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters). You are printing the path over the terrain.
This is an impure function (it does not return anything).

Note that in Python:
```
> a = ["*", "-", "*"]
> "".join(a)
*-*
```
Do not print raw data structures; do not insert unneeded/requested spaces!

### Additional comments

As Python is an interpreted language, you're going to need to insert all of your functions *before* the actual `a_star_search` function implementation. 
Do not make unwarranted assumptions (for example, do not assume that the start is always (0, 0).
Do not refer to global variables, pass them as parameters (functional programming).

Simple and correct is better than inefficient and incorrect, or worse, incomplete.
For example, you can use a simple List, with some helper functions, as a Stack or a Queue or a Priority Queue.
Avoid the Python implementations of HeapQ, PriorityQueue implementation unless you are very sure about what you're doing as they require *immutable* keys.

In [373]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy
from queue import PriorityQueue
import numpy as np

*add as many markdown and code cells here as you need for helper functions. We have added `heuristic` for you*

## get_neighbors

`get_neighbors` looks at the current node and gets its neighbors that are within the world space **Used by**: [a_star_search](#a_star_search)

* **position** tuple: the position of the current node.
* **moves**: list: list of tuples that contain all the possible action moements in the world.
* **world**: list: list of lists that contain the mapping of the world.

**returns** List: list of tuples that states the current node's neighbors.

In [374]:
def get_neighbors(position: tuple, moves: List[Tuple[int, int]], world: List[List[str]]) -> list:
    
    possible_neighbors = [tuple(np.subtract(position, move))
                          for move in moves]
    neighbors = []
    # Checks if it is within the world and then appends to actual neighbors
    for neighbor in possible_neighbors:
        if (neighbor[0] in range(len(world)) and 
            neighbor[1] in range(len(world[0]))):
            neighbors.append(neighbor)
    return neighbors 

In [375]:
# assertions/unit tests
start = (0,0)
assert get_neighbors(start,MOVES,test_world) == [(0,1), (1,0)]
start = (0,3)
assert get_neighbors(start,MOVES,test_world) == [(0,4), (0,2), (1,3)]
start = (7,7)
assert get_neighbors(start,MOVES,test_world) == []

<a id="heuristic"></a>
## heuristic
`heuristic` - In this world, the only movements allowed are left, right, up and down, therefor the best type of heuristic function for this would be using the Mahattan Distance which is defined as h(n) = | x-i | + | y-j |. **Used by**: [a_star_search](#a_star_search)

f(n) = g(n) + h(n) 
* **h(n)** : Heuristic from Greedy Search, estimated cost of the cheapest path from the state at node n to goal state 
* **g(n)** : Path cost from the initial State to node n (cost of path taken so far)
* **f(n)** : estimated cost of the best path that continues 

**returns** int : the heuristic value h(n)

In [376]:
def heuristic(start: tuple, goal: tuple) -> int:
    h = abs(start[0] - goal[0]) + abs(start[1] - goal[1])
    return h 

In [377]:
# assertions/unit tests
start = (0,0)
assert heuristic(start,(3,3)) == 6
start = (0,0)
assert heuristic(start,(2,2)) == 4
start = (3,3)
assert heuristic(start,(3,3)) == 0

<a id="a_star_search"></a>
## a_star_search
`a_star_search` is a best-first search algorithm that uses f(n) = g(n) + h(n). Whether `a_star_search` is cost-optimal depends on the ley property that the `heuristic` is admissible, an admissible heuristic is one that never overestimates the cost to reach the goal. The algorithm starts off in the Initial State, at the frontier it then gets the neighbors and calculates the f(n), the estimated cost of the best path that it continues. After evaluating all the neighbors, the one with the lowest cost will then be pushed into the priority queue. Now the next state in the priority queue is evaluated and etc. The algorithm continues until it reaches the goal State. 

* **h(n)** : Heuristic from Greedy Search, estimated cost of the cheapest path from the state at node n to goal state 
* **g(n)** : Path cost from the initial State to node n (cost of path taken so far)
* **f(n)** : estimated cost of the best path that continues 

* **world** List[List[str]]: the actual context for the navigation problem.
* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.
* **heuristic** Callable: is a heuristic function that returns an estimate of the total cost $f(x)$ from the start to the goal through the current node, $x$. The heuristic function might change with the movement model. EDITED: Estimated cost of getting from n to the goal


**returns** List[Tuple[int, int]]: the offsets needed to get from start state to the goal as a `List`.


In [378]:
def a_star_search( world: List[List[str]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int], moves: List[Tuple[int, int]], heuristic: Callable) -> List[Tuple[int, int]]:
    frontier = PriorityQueue()
    frontier.put(start, 0)
    
    path = {start: None}
    g_n = {start: 0}
    while not frontier.empty():
        current_node = frontier.get()
        if current_node == goal:
            final = [current_node]
            while current_node in path:
                current_node = path[current_node]
                final.append(current_node)
            final = final[::-1]
            path_actions = [tuple(np.subtract(j,i)) for i, j in zip(final[1:], final[2:])]
            final_path_actions = []
            for action in path_actions:
                reversed_action = action[::-1]
                final_path_actions.append(reversed_action)
            break
        neighbors = get_neighbors(current_node, moves, world)
        for neighbor in neighbors:

            new_g = g_n[current_node] + costs.get(world[neighbor[0]][neighbor[1]])
    
            if neighbor not in g_n or new_g < g_n[neighbor]:
                g_n[neighbor] = new_g
                priority = new_g + heuristic(neighbor, goal)
                frontier.put(neighbor,priority)
                path[neighbor] = current_node
            
    return final_path_actions

## pretty_print_path

`pretty_print_path` is a function used to print out the world and demonstrate the path returned from `a_star_search` This function also uses the cost dictionary along with the path's action tuples to calculated the total path cost. 

* **world** List[List[str]]: the world (terrain map) for the path to be printed upon.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.
* **start** Tuple[int, int]: the starting location for the path.
* **goal** Tuple[int, int]: the goal location for the path.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.

**returns** int - The path cost.


<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        Does your output of pretty_print_path really look like the specification? Go check again.
    </p>
</div>

In [379]:
def pretty_print_solution( world: List[List[str]], path: List[Tuple[int, int]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int] ) -> int:
    ### YOUR SOLUTION HERE ###
    dictionary = {(0,1):'⏬', (1,0):'⏩', (0,-1):'⏫', (-1,0):'⏪', goal:'🎁'}
    
    total_cost = 0
    for action in path:
            world[start[1]][start[0]] = dictionary.get(action)
            start = tuple(np.add(start, action))
            total_cost += costs.get(world[start[1]][start[0]])
            world[start[1]][start[0]] = dictionary.get(action)
            
    world[goal[1]][goal[0]] = dictionary.get(goal)     
    
    for row in world:
        row_string = ''
        for char in row:
            row_string += f'{str(char):<1} '.replace('.', ' ')
        print(row_string)
    return total_cost # replace with the real value!

## Problems

## Problem 1. 

Execute `a_star_search` and `print_path` for the `test_world`.

If you change any values while developing your code, make sure you change them back! (Better yet, don't do it. Copy them elsewhere and change the values, then delete those experiments).

In [380]:
test_start = (0, 0)
test_goal = (len(test_world[0]) - 1, len(test_world) - 1)
test_path = a_star_search(test_world, test_start, test_goal, COSTS, MOVES, heuristic)
test_path_cost = pretty_print_solution(test_world, test_path, test_start, test_goal, COSTS)
print(f"total path cost: {test_path_cost}")
print(test_path)

⏬ 🌲 🌲 🌲 🌲 🌲 🌲 
⏬ 🌲 🌲 🌲 🌲 🌲 🌲 
⏬ 🌲 🌲 🌲 🌲 🌲 🌲 
⏩ ⏩ ⏩ ⏩ ⏩ ⏩ ⏬ 
🌲 🌲 🌲 🌲 🌲 🌲 ⏬ 
🌲 🌲 🌲 🌲 🌲 🌲 ⏬ 
🌲 🌲 🌲 🌲 🌲 🌲 🎁 
total path cost: 12
[(0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1)]


## Problem 2

Execute `a_star_search` and `print_path` for the `full_world`

In [381]:
full_start = (0, 0)
full_goal = (len(full_world[0]) - 1, len(full_world) - 1)
full_path = a_star_search(full_world, full_start, full_goal, COSTS, MOVES, heuristic)
full_path_cost = pretty_print_solution(full_world, full_path, full_start, full_goal, COSTS)
print(f"total path cost: {full_path_cost}")
print(full_path)

⏬ 🌾 🌾 🌾 🌾 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 🌾 
⏬ 🌾 🌾 🌾 🌾 🌾 🌾 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌾 🌾 🗻 🗻 🗻 🗻 🗻 🗻 🗻 🌾 🌾 
⏬ 🌾 🌾 🌾 🗻 🗻 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🗻 🗻 🗻 🪨 🪨 🪨 🗻 🗻 🪨 🪨 
⏬ 🌾 🌾 🌾 🪨 🗻 🗻 🗻 🌲 🌲 🌲 🌲 🐊 🐊 🌲 🌲 🌲 🌲 🌲 🌾 🌾 🪨 🪨 🗻 🗻 🪨 🌾 
⏬ 🌾 🌾 🪨 🪨 🗻 🗻 🌲 🌲 🌾 🌾 🐊 🐊 🐊 🐊 🌲 🌲 🌲 🌾 🌾 🌾 🪨 🗻 🗻 🗻 🪨 🌾 
⏬ 🪨 🪨 🪨 🗻 🗻 🪨 🪨 🌾 🌾 🌾 🌾 🐊 🐊 🐊 🐊 🐊 🌾 🌾 🌾 🌾 🌾 🪨 🗻 🪨 🌾 🌾 
⏬ 🪨 🪨 🗻 🗻 🪨 🪨 🌾 🌾 🌾 🌾 🪨 🗻 🗻 🗻 🐊 🐊 🐊 🌾 🌾 🌾 🌾 🌾 🪨 🌾 🌾 🌾 
⏬ 🌾 🪨 🪨 🪨 🪨 🪨 🌾 🌾 🌾 🌾 🌾 🌾 🪨 🗻 🗻 🗻 🐊 🐊 🐊 🌾 🌾 🪨 🪨 🪨 🌾 🌾 
⏬ 🌾 🌾 🪨 🪨 🪨 🌾 🌾 🌾 🌾 🌾 🌾 🪨 🪨 🗻 🗻 🌾 🐊 🐊 🌾 🌾 🪨 🪨 🪨 🌾 🌾 🌾 
⏬ 🌾 🌾 🐊 🐊 🐊 🌾 🌾 🪨 🪨 🪨 🗻 🗻 🗻 🗻 🌾 🌾 🌾 🐊 🌾 🪨 🪨 🪨 🌾 🌾 🌾 🌾 
⏬ 🌾 🐊 🐊 🐊 🐊 🐊 🌾 🪨 🪨 🗻 🗻 🗻 🪨 🌾 🌾 🌾 🌾 🌾 🪨 🗻 🗻 🗻 🪨 🌾 🌾 🌾 
⏬ 🐊 🐊 🐊 🐊 🐊 🌾 🌾 🪨 🗻 🗻 🪨 🌾 🌾 🌾 🌾 🐊 🐊 🌾 🌾 🪨 🗻 🗻 🪨 🌾 🌾 🌾 
⏬ 🐊 🐊 🐊 🐊 🌾 🌾 🪨 🪨 🗻 🗻 🪨 🌾 🐊 🐊 🐊 🐊 🌾 🌾 🌾 🪨 🗻 🪨 🌾 🌾 🌾 🌾 
⏬ 🐊 🐊 🐊 🐊 🌾 🌾 🪨 🌲 🌲 🪨 🌾 🌾 🌾 🌾 🐊 🐊 🐊 🐊 🌾 🌾 🪨 🌾 🌾 🌾 🌾 🌾 
⏩ ⏬ 🌾 🌾 🗻 🌾 🌾 🌲 🌲 🌲 🌲 🪨 🪨 🪨 🪨 🌾 🐊 🐊 🐊 🌾 🌾 🪨 🗻 🪨 🌾 🌾 🌾 
🌾 ⏬ 🌾 🗻 🗻 🗻 🌲 🌲 🌲 🌲 🌲 🌲 🗻 🗻 🗻 🪨 🪨 🌾 🐊 🌾 🪨 🗻 🗻 🪨 🌾 🌾 🌾 
🌾 ⏬ 🗻 🗻 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🗻 🗻 🗻 🌾 🌾 🗻 🗻 🗻 🌾 🌾 🌾 🌾 🌾 
🌾 ⏬ 🌾 🗻 🗻 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🌲 🗻 🗻 🗻 🗻 🌾 🌾 🌾 🌾 🌾 🌾 🌾 
🌾 ⏬ 🌾 🗻 🗻 

## Comments

needed to handle the test case for the implausible mountain in problem 2. Also then just needed to work on the mapping of the path to the ASCII output 

EDITED: Finally figured it out

## To think about for future assignments...

This first assignment may not have been difficult for you if you've encountered A* search before in your Algorithms course. In preparation for future assignments that build on State Space Search, you can think about the following or even do an implementation if you like. You should **not** submit it as part of this assignment.

In several future assignments, we will have a need for a "plain ol'" Depth First Search algorithm.

1. Implement DFS Search to solve the problem presented in this programming assignment. Try to be as general as possible (don't hard code anything you can pass as a formal parameter).
2. Can you implement DFS Search as a higher order function and supply your own `is_goal`, `successors`, and `path` functions? How do you handle *state*?
3. Can you write a version of DFS that returns all the solutions?

In one future assignment a Breadth First Search algorithm will be very handy. Can you implement a search algorithm that changes whether it uses DFS or BFS by parameterization?

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you follow the Programming Requirements on Canvas?

Do not submit any other files.